## 项目背景

文本是最常用的序列数据之一，可以理解为字符序列或单词序列，但最常见的是单词级处理。深度学习序列处理模型都可以根据文本生成基本形式的自然语言理解，并可用于文档分类、情感分析、作者识别甚至问答（QA，在有限的语境下）等应用。当然，这些深度学习模型都没有像人类一样真正地理解文本，而只是映射出书面语言的统计结构，但这足以解决许多简单的文本分类任务。深度学习用于自然语言处理是将模式识别应用于单词、句子和段落，这与计算机视觉将模式识别应用于像素大致相同。
文本分类不仅仅可以针对于英文，也可针对于中文进行分类，中文与英文的区别在于中文需要进行分词。

## 项目步骤

1. 数据处理：对数据进行预处理，包括对数据进行分词与停词过滤的操作。
2. 建立模型：建立逻辑回归模型。
3. 模型训练与评估：使用训练数据集进行训练，并使用测试集进行评估。



In [1]:
pip install jieba

Note: you may need to restart the kernel to use updated packages.


In [10]:
!ls /home/jovyan/0f0bbc7e-e73a-49d7-be82-1fe790e44cae.ipynb

0f0bbc7e-e73a-49d7-be82-1fe790e44cae.ipynb  data  __MACOSX  work


1. 读取数据，训练集与测试集，并对数据进行分词操作。
2. 对数据进行停词过滤，比如数据中的”的“，”是“等这样的词显然对我们的分析和统计没有什么帮助，反而有的时候会干扰我们的统计，当然我们需要把这些没有用的词给去掉，现在有很多可以去除这些词的方法，比如使用一些停用词的语料库等。



In [2]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
# 从Scikit-learn中导入逻辑回归算法库，补全此处代码
from sklearn.linear_model import LogisticRegression
# 从Scikit-learn中导入划分数据集的库，补全此处代码
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np 
import jieba

# 使用pandas的read_csv方法读取训练集数据，训练集位于"/home/jovyan/data/7-text_classification"文件夹中，名称为cnews.train.txt，数据以回车分行，补全此处代码
train_df = pd.read_csv('/home/jovyan/data/7-text_classification/cnews.train.txt', sep='\t',header=None)
# 使用pandas的read_csv方法读取测试集数据，训练集位于"/home/jovyan/data/7-text_classification"文件夹中，名称为cnews.train.txt，数据以回车分行，补全此处代码
test_df = pd.read_csv('/home/jovyan/data/7-text_classification/cnews.test.txt', sep='\t',header=None)

#或者通过columns来查看
train_df.columns = ['Subject', 'Content']
train_df['Subject'].value_counts().sort_index()
print(train_df)

# 分词
def cut_context(data):
   # 采用全模式的方法对数据进行分词，补全此处代码
    words=data.apply(lambda x: ' '.join(jieba.cut(x)))
    return words

#停词过滤
stopwords=open('/home/jovyan/data/7-text_classification/cnews.vocab.txt',encoding='utf-8')
stopwords_list=stopwords.readlines()
stopworsd=[x.strip() for x in stopwords_list]#去掉每行头尾空白 

#使用TfidfVectorizer方法原始文档集合转换为TF-IDF功能矩阵，要求所有停用词需要从转换结果中删除，最大特征词汇表为5000，在转换前不需要将所有字符转换为小写
tfidf=TfidfVectorizer(stop_words=stopwords,max_features=5000,lowercase=False)
X = tfidf.fit_transform(cut_context(train_df['Content']))
print('词表大小', len(tfidf.vocabulary_))
print('数据大小',X.shape)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
      Subject                                            Content
0          体育  马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1          体育  商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2          体育  冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3          体育  辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4          体育  揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道  谢亚龙已经被公安...
...       ...                                                ...
49995      财经  打好投资的决胜局□国泰基金 阿邰就长期而言，资产配置占投资成功与否的决定因素高达90%以上。...
49996      财经  昔明星基金今年首月负收益 QDII成今年胸闷基金□晨报记者 陈重博2009年以来，所有偏股型...
49997      财经  沪基指全周大涨8.25% 创两个月最大周涨幅全景网2月6日讯 受到A股市场牛年高歌猛进影响，...
49998      财经  沪基指半日涨2.01% 两市封基近乎全线上扬全景网2月6日讯 沪深基金指数周五早盘大幅收高，...
49999      财经  牛年第一月 开基抬头券商集合理财掉队每经记者 于春敏在金融危机的淫威之下，2008年，全球资...

[50000 rows x 2 columns]
Loading model cost 0.700 seconds.
Prefix dict has been built successfully.
词表大小 5000
数据大小 (50000, 5

1. 建立逻辑回归模型，并使用训练集进行训练。
2. 使用测试集对模型进行测试，得出其预测的准确率。



In [6]:
#建立模型
#标签编码

train_df = pd.read_csv('/home/jovyan/data/7-text_classification/cnews.train.txt', sep='\t', header=None)
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(train_df[0])#一旦给train_df加上columns，就无法使用[0]来获取第一列了

#逻辑回归
# 使用train_test_split方法将数据集划分为训练集与测试集，按照8：2的比例进行部分，补全此处代码
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.2)
# 使用LogisticRegression算法创建逻辑回归模型，多类选项采用multinomial，迭代优化损失函数采用lbfgs算法，补全此处代码
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# 使用fit方法训练逻辑回归模型，补全此处代码
logistic_model.fit(train_X, train_y)
# 使用score方法对训练后的逻辑回归模型对于测试集进行预测，并得出预测准确率，补全此处代码
logistic_model.score(test_X,test_y)


cv_split = ShuffleSplit(n_splits=5, test_size=0.3)
score_ndarray = cross_val_score(logistic_model, X, y, cv=cv_split)
print(score_ndarray)
print(score_ndarray.mean())

[0.9608     0.95966667 0.96113333 0.9614     0.96093333]
0.9607866666666667


从预测精确率，召回率，F1分数等方面评估模型的好坏。



In [0]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
logistic_model = LogisticRegressionCV(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(train_X, train_y)
predict_y = logistic_model.predict(test_X)
 
pd.DataFrame(confusion_matrix(test_y, predict_y),columns=labelEncoder.classes_, index=labelEncoder.classes_)

#绘制precision、recall、f1-score、support报告表： 
def eval_model(y_true, y_pred, labels):
    #计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support( y_true, y_pred)
    #计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision' : p,
        u'Recall' : r,
        u'F1' : f1,
        u'Support' : s
    })
     
    res2 = pd.DataFrame({
        u'Label' : ['总体'],
        u'Precision' : [tot_p],
        u'Recall': [tot_r],
        u'F1' : [tot_f1],
        u'Support' : [tot_s]
    })
     
    res2.index = [999]
    res = pd.concat( [res1, res2])
    return res[ ['Label', 'Precision', 'Recall', 'F1', 'Support'] ]
 
predict_y = logistic_model.predict(test_X)
eval_model(test_y, predict_y, labelEncoder.classes_)
# 使用pandas的read_csv方法读取验证集数据，训练集位于"/home/jovyan/data/7-text_classification"文件夹中，名称为cnews.val.txt，数据以回车分行，补全此处代码
test_df = pd.read_csv('/home/jovyan/data/7-text_classification/cnews.val.txt',sep='\t',header=None)
test_X = tfidf.transform(cut_context(test_df[1]))
test_y = labelEncoder.transform(test_df[0])
# 使用predict方法对验证集进行预测，补全此处代码
predict_y = logistic_model.predict(test_X)
# 调用eval_model方法，输出验证集的报告表，补全此处代码
eval_model(test_y,predict_y,labelEncoder.classes_)